In [1]:
#%matplotlib qt
from __future__ import print_function

#import rospy
#import rosbag
import itertools
from  tqdm import tqdm
#from geometry_msgs.msg import PoseWithCovarianceStamped
#import tf
import numpy as np

import matplotlib.pyplot as plt
import os
import time

%load_ext autoreload
%autoreload 2

In [3]:
import GPy

#import wifi-localization

from Localization.sensormodel.gp import GP
#from Localization.sensormodel.hybrid import hGP #これはcommentでも良い
#Localizetion.sensormodel.gp→みやぐすく先生のgitから！

In [4]:
# instead of rss.classes import ProcessedData
class ProcessedData():
    def __init__(self,X,Z,mac_address_dict):
        self.data = {'X':X, 'Y':Z, 'Var':Z}
        self.nm = X.shape[0]
        self.all_mac_dict = mac_address_dict

In [8]:
save_model = True #いっちゃん下のif文に関係あるお

#root_folder = '/mnt/matrix/rosbag/processed_data/nict'
root_folder = '/home/hiro/data'
map_name = 'udai-01'


print('\n####################\n  map: {:s} \n####################'.format(map_name)); time.sleep(0.2)
save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_WifiData.npz'.format(root_folder,map_name)

print('Load data')
data = np.load(save_filename,allow_pickle=True) #wifiData.npzを配列のままloadして、dataに格納
XY = data['XY'] #X, Y座標をndarray型[X, Y]で格納
Wifi = data['Wifi'] #wifiのデータをWifiに格納
macdict = data['macdict'].item() #macdictにmacaddresを格納

print('Wifi  : '  ,data['Wifi'].shape,
      '\nXY    : ',data['XY'].shape)

print('Create model') 
traindata = ProcessedData(XY,Wifi,macdict) #ProcessedDataクラスをインスタンス化

# Train model
print('Training GP')
time.sleep(0.2)

kernel = GPy.kern.RBF(2)#+GPy.kern.RBF(2,lengthscale=5)
#kernel = GPy.kern.Matern32(2)

t0 = time.time()
model = GP(traindata.data,all_mac_dict=macdict) #GPモデルをインスタンス化
model.optimize(kernel=kernel) #最適化の実行
print(time.time()-t0)

print(model)

if save_model:
    save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_WifiModel.npz'.format(root_folder,map_name)
    #save_filename = f'../datadir/{map_name}_WifiModel.npz'
    print('Saving model',save_filename)
    np.savez_compressed(save_filename,param_array=model.gp.param_array)


####################
  map: udai-01 
####################
Load data
Wifi  :  (2288, 270) 
XY    :  (2288, 2)
Create model
Training GP
18.339536666870117
Name                                : GP
data
  X                                 : [2288,   2]
  Y                                 : [2288, 270]
  Var                               : [2288, 270]

Gaussian Process
  Name : GP regression
  Objective : -982693.7303278127
  Number of Parameters : 3
  Number of Optimization Parameters : 3
  Updates : True
  Parameters:
    GP_regression.           |                  value  |  constraints  |  priors
    rbf.variance             |   0.008158355749386064  |      +ve      |        
    rbf.lengthscale          |      8.054248495626641  |      +ve      |        
    Gaussian_noise.variance  |  0.0020024063420131873  |      +ve      |        
Saving model /home/hiro/data/udai-01/invisible-maps/udai-01_WifiModel.npz


# Create Map from trained model

In [11]:
# Wifi Map
res = 0.25

print('Creating Wifi map, res {:.2f}'.format(res))
time.sleep(0.2)

xmin,xmax = np.min(data['XY'][:,0]), np.max(data['XY'][:,0])
ymin,ymax = np.min(data['XY'][:,1]), np.max(data['XY'][:,1])

x = np.arange(np.floor(xmin-20),np.ceil(xmax+20),res)
y = np.arange(np.floor(ymin-20),np.ceil(ymax+20),res)
print(x.shape, y.shape) 
xy = np.meshgrid(x,y) #配列から格子データ生成 
XYtest = np.concatenate([xy[0].reshape(-1,1),xy[1].reshape(-1,1)],axis=1).astype(np.float32) #x, y座標を一列に直す

batch_eval = 5000 #batchsize
iterations = int(np.ceil(XYtest.shape[0]/batch_eval)) #iteration数

Predictions_mean = np.zeros((XYtest.shape[0],Wifi.shape[1])) #平均値の予測値を格納するための配列初期化
Predictions_var = np.zeros((XYtest.shape[0],1)) #分散値の予測値を格納するための配列初期化

# Make predictions
for i in tqdm(range(iterations+1)):
    try:
        xy = XYtest[i*batch_eval:(i+1)*batch_eval]
        #print('xy in mem')
        predictions_mean,predictions_variance = model.predict(xy) 
        #print('predictions')
        Predictions_mean[i*batch_eval:(i+1)*batch_eval,:] = predictions_mean
        Predictions_var[i*batch_eval:(i+1)*batch_eval,:]  = predictions_variance  
        #print('copied to cpu')
    except:
        xy = XYtest[i*batch_eval:-1]
        predictions_mean,predictions_variance = model.predict(xy)
        Predictions_mean[i*batch_eval:-1,:] = predictions_mean
        Predictions_var[i*batch_eval:-1]    = predictions_variance

print(f"type of Mean {type(Predictions_mean)}")
print(f"type of Var {type(Predictions_var)}")
print(f"shape of Mean {Predictions_mean.shape}")
print(f"type of Var {Predictions_var.shape}")

print("============================================")
print(f"predicton_mean = {predictions_mean}")
print(f"predicton_variance =  {predictions_variance}")
#print(f"model predict = {model.predict(xy)}")


#np.savez_compressed("../datadir/test.npz",Mean=Predictions_mean, Var=Predictions_var,macdict=macdict)
print(Predictions_mean)
print("====")
print(Predictions_var)
      
# Save data
if save_model: 
    save_filename = '{0:s}/{1:s}/invisible-maps/{1:s}_WifiMap_res_{2:03d}.npz'.format(root_folder, map_name,int(res*100))
    #save_filename = f'../datadir/{map_name}_WifiMap_res_{int(res*100):03d}.npz'
    np.savez_compressed(save_filename,Mean=Predictions_mean,Var=Predictions_var,macdict=macdict)
    print(f"Saved! {save_filename}")

Creating Wifi map, res 0.25
(924,) (616,)


100%|██████████| 115/115 [00:57<00:00,  1.99it/s]


type of Mean <class 'numpy.ndarray'>
type of Var <class 'numpy.ndarray'>
shape of Mean (569184, 270)
type of Var (569184, 1)
predicton_mean = []
predicton_variance =  []
[[0.00000000e+00 6.29940657e-20 0.00000000e+00 ... 4.03639219e-20
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.84292581e-20 0.00000000e+00 ... 4.47919852e-20
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.42527209e-20 0.00000000e+00 ... 4.96463122e-20
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 1.32707434e-05 1.07523383e-05 ... 0.00000000e+00
  0.00000000e+00 7.86190326e-09]
 [0.00000000e+00 1.20487157e-05 9.47929590e-06 ... 0.00000000e+00
  0.00000000e+00 6.97794660e-09]
 [0.00000000e+00 1.09286751e-05 8.34592764e-06 ... 0.00000000e+00
  0.00000000e+00 6.18654412e-09]]
====
[[0.01066076]
 [0.01066076]
 [0.01066076]
 ...
 [0.01066076]
 [0.01066076]
 [0.01066076]]
Saved! /home/hiro/data/udai-01/invisible-maps/udai-01_WifiMap_res_025.npz
